In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib
import Crypto
from Crypto.PublicKey import RSA
from Crypto import Random
import base64

In [24]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['payer', 'payee', 'amount'])
Puzzle = namedtuple('Puzzle', ['difficulty', 'denominator', 'numerator'])
User = namedtuple('User', ['secret_key', 'public_key'])

class Block:
    def __init__(self, transaction, prev, nonce):
        self.transaction = transaction
        self.prev = prev
        self.nonce = nonce
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transaction},\n nonce: {self.nonce},\n prev: {self.prev})'  
    
# Need this outside class since we need to make the users before making the Blockchain
def new_user():  
    length=1024  
    privatekey = RSA.generate(length, Random.new().read)  
    publickey = privatekey.publickey()  
    return privatekey, publickey

class Blockchain:
    def __init__(self, payer, payee, amount):
        first_transaction = Transaction(payer, payee, amount)
        self.blockchain = hashlib.sha256(bytes(str(Block(first_transaction, None, None)), encoding='utf-8')).hexdigest()
        self.users = {payer: amount, payee:amount}
        self.puzzle = self.new_puzzle()
        
        return_value = self.process_transaction(first_transaction)
        print(return_value)
        assert return_value == ''      
    
    def sign(self, sk, m):
        return base64.b64encode(str((privatekey.sign(m,''))[0]).encode())
    
    def verify_signature(self, pk, sig, m):
        return publickey.verify(m,(int(base64.b64decode(sig)),))
        
    def process_transaction(self, transaction):
        users = self.users
        payer = transaction.payer
        payee = transaction.payee
        amount = transaction.amount
        
        return_value = ''
        
        if payer == None and payee != None:
            if payee not in users.keys():
                users[payee] = amount
            else:
                return_value = 'This user already exists in the system.'
        elif payer != None and payee != None:
            if payee in users.keys() and payer in users.keys():
                payer_balance = users[payer]
                payee_balance = users[payee]
                
                if payer_balance - amount < 0:
                    return_value += 'Payer does not have enough currency. '
                else:
                    users[payer] = payer_balance - amount
                    users[payee] = payee_balance + amount
                    
            else:
                if payee not in users.keys():
                    return_value += 'Payee does not exist. '
                if payer not in users.keys():
                    return_value += 'Payer does not exist. '
        elif payee == None:
            return_value += 'Payee not specified. '   
                
        self.users = users
        return return_value
        
        
    def get_blockchain(self):
        return self.blockchain

    # Creates a block with the given transaction
    def add_transaction(self, transaction, nonce):
        prev_hash = hashlib.sha256(bytes(str(self.blockchain), encoding='utf-8')).hexdigest()
        prev = HashPointer(prev_hash, self.blockchain)
        new_block = Block(transaction, prev, nonce)
        self.blockchain = new_block
        
        block_hash = hashlib.sha256(bytes(str(new_block), encoding='utf-8')).hexdigest()

        return new_block, block_hash
    
    def new_puzzle(self):
        try:
            self.puzzle
        except AttributeError:
            denominator = 1  
        else:
            denominator = self.puzzle.denominator     
        
        hash_string = hashlib.sha256(bytes('hello', encoding='utf-8')).digest()
        denominator = np.random.randint(denominator, denominator*17)
        numerator = 2**(len(hash_string) * 8)
        difficulty = int(numerator/denominator)
        return Puzzle(difficulty, denominator, numerator)
        
    def mine(self, transaction):
        blockchain = self.blockchain
        nonce = 0
        finished = False
        
        while not finished:
            nonce = nonce + 1
            new_blockchain, final_hash = self.add_transaction(transaction, nonce)
            
            if self.verify_puzzle_solution(final_hash):
                self.blockchain = new_blockchain
                self.puzzle = self.new_puzzle()
                finished = True
                
        
    def verify_puzzle_solution(self, final_hash):
        difficulty = self.puzzle.difficulty
        
        if int(final_hash, 16) <= difficulty:
            return True
        return False
        

    def get_puzzle(self):
        return self.puzzle
    

In [25]:
# List of possible test cases
# 1: Sending / Receiving Transactions
#  1a: Verify Transaction (Check for: Double Payee Entry, Doesnt Exist, Not Specified)
#  1b: Check balances
#  1c: Check for double spending
# 2: Mining
#  2a: Push solution to Blockchain
#  2b: Verify if solution is correct (dont inside function)
#  2c: Add mined block to blockchain (gonna be tough since we hash the blockchain)
# etc ...

In [26]:
import unittest

Matt = User
Luke = User
Matt.secret_key, Matt.public_key = new_user()
Luke.secret_key, Luke.public_key = new_user()
b = Blockchain(Matt, Luke, 2000)

class BlockchainTests(unittest.TestCase):
        
    def tearDown(self):
        pass
    
    def test_VerifyTransaction(self):
        self.assertEqual('','')
    def test_InvalidBalance(self):
        self.assertEqual(1,1)
    def test_AttemptDoubleSpending(self):
        self.assertEqual(1,1)
        
    # Will check if you can succesfully mine 
    def test_SolvePuzzle(self):
        first_puzzle = b.get_puzzle()
        b.mine(Transaction(Matt, None, 1))
        self.assertNotEqual(first_puzzle, b.get_puzzle())
    
    # Blockchain will change if block was added
    def test_AddBlockAfterMining(self):
        first_blockchain = b.get_blockchain()
        b.mine(Transaction(Matt, None, 1))
        self.assertNotEqual(first_blockchain, b.get_blockchain())
        

In [27]:
# Will run all the test cases and show which passed and which didnt
unittest.main(argv=[''], verbosity=2, exit=False)

test_AddBlockAfterMining (__main__.BlockchainTests) ... ok
test_AttemptDoubleSpending (__main__.BlockchainTests) ... ok
test_InvalidBalance (__main__.BlockchainTests) ... ok
test_SolvePuzzle (__main__.BlockchainTests) ... ok
test_VerifyTransaction (__main__.BlockchainTests) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.173s

OK
